In [135]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

In [136]:
from io import SEEK_CUR
import pandas as pd
import numpy as np

import dash
from datetime import date
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
import plotly.graph_objects as go
import plotly.express as px

from PIL import ImageColor

from dash.dependencies import Input, Output, State

from dashboard_main import *

In [137]:

features_forecast = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'WS100', 'WD100']
features_retro = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'TARGETVAR']

PATH_PREDICTIONS = 'RandomForest_Predictions.csv'
PATH_DATA_ALL = 'raw_data_incl_features_test.csv'

initial_date = '2013-07-01'

In [138]:
data_forecast = pd.read_csv(PATH_PREDICTIONS, parse_dates=['TIMESTAMP'])
data_forecast.rename(
    columns={x: 'Wind Farm '+x.split()[-1] 
    for x in data_forecast.columns if x.startswith('Zone')}, 
    inplace=True)
add_HOUR_column(data_forecast)
data_all = pd.read_csv(PATH_DATA_ALL, parse_dates=['TIMESTAMP'])
data_all.interpolate(method='bfill', inplace=True)
data_all['ZONEID'] = data_all['ZONEID'].apply(lambda x: 'Wind Farm '+str(x))
data_all['HOUR'].replace(0,24, inplace=True)
data_all.replace('Zone', 'Wind Farm', inplace=True)
data_all.reset_index(inplace=True)

In [116]:
day = '2013-08-15'
zone = 'Wind Farm 8'

In [117]:
delta_minute = pd.Timedelta(1, unit='min')

In [118]:
data_forecast['TIMESTAMP'] = data_forecast['TIMESTAMP'].apply(lambda x: x-delta_minute)
data_forecast.drop('HOUR', axis=1, inplace=True)

In [119]:
data_forecast = data_forecast[['TIMESTAMP', zone]]

In [120]:
data_forecast['DATE'] = data_forecast['TIMESTAMP'].dt.date 

In [121]:
data_forecast = data_forecast.groupby('DATE').mean()

In [122]:
data_forecast_mean = data_forecast.mean(axis=1)
data_forecast_mean
df_forecast_mean = pd.DataFrame(data_forecast_mean)

In [123]:
df_forecast_mean.columns = ['TARGETVAR_PREDICTED']
df_forecast_mean

,TARGETVAR_PREDICTED
DATE,
2013-07-01,0.600115
2013-07-02,0.563627
2013-07-03,0.584431
2013-07-04,0.732470
2013-07-05,0.724957
...,...
2013-12-27,0.299677
2013-12-28,0.380428
2013-12-29,0.091375


## observed values

In [124]:
data_all

,index,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,IS_HOLIDAY,WS10,WS100,WD10,WD100,WD100CARD,WD10CARD,U100NORM,V100NORM
0,0,Wind Farm 1,2013-06-30 01:00:00,0.375811,4.519248,-1.559637,6.913626,-2.293687,1,6,6,0,4.780803,7.284176,289.040056,288.353946,WNW,WNW,0.949129,-0.314886
1,1,Wind Farm 1,2013-06-30 02:00:00,0.519688,4.810528,-1.128186,6.989617,-1.628825,2,6,6,0,4.941051,7.176895,283.198721,283.117810,WNW,WNW,0.973905,-0.226954
2,2,Wind Farm 1,2013-06-30 03:00:00,0.520346,4.560787,-0.831092,6.590505,-1.121018,3,6,6,0,4.635892,6.685165,280.327442,279.653384,W,W,0.985840,-0.167687
3,3,Wind Farm 1,2013-06-30 04:00:00,0.428907,4.875876,-0.787637,7.088597,-1.173369,4,6,6,0,4.939083,7.185054,279.176149,279.398894,W,W,0.986575,-0.163307
4,4,Wind Farm 1,2013-06-30 05:00:00,0.665915,4.956422,-0.793845,7.314015,-1.150858,5,6,6,0,5.019592,7.404005,279.099489,278.942154,W,W,0.987846,-0.155437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44395,44395,Wind Farm 10,2013-12-31 20:00:00,NaN,1.032363,-6.281558,2.041033,-11.220655,20,12,1,0,6.365826,11.404776,350.666975,349.690637,N,N,0.178963,-0.983856
44396,44396,Wind Farm 10,2013-12-31 21:00:00,NaN,1.702361,-6.202448,2.846245,-10.486079,21,12,1,0,6.431827,10.865494,344.652195,344.814040,NNW,NNW,0.261953,-0.965081
44397,44397,Wind Farm 10,2013-12-31 22:00:00,NaN,5.086629,-1.261378,7.382256,-3.097656,22,12,1,0,5.240694,8.005821,283.927218,292.763328,WNW,WNW,0.922111,-0.386925
44398,44398,Wind Farm 10,2013-12-31 23:00:00,NaN,4.183751,-1.580172,5.789054,-2.116548,23,12,1,0,4.472216,6.163840,290.691183,290.083020,WNW,WNW,0.939196,-0.343381


In [134]:
# data_all = data_all[data_all['ZONEID'] == zone]
data_all

,index,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,...,IS_HOLIDAY,WS10,WS100,WD10,WD100,WD100CARD,WD10CARD,U100NORM,V100NORM,DATE
25872,25872,Wind Farm 8,2013-06-30 00:59:00,0.521840,5.213761,-0.560161,7.682476,-0.732571,1,6,...,0,5.243766,7.717324,276.132271,275.447033,W,W,0.995484,-0.094926,2013-06-30
25873,25873,Wind Farm 8,2013-06-30 01:59:00,0.722472,6.079496,-0.769900,8.729746,-1.067619,2,6,...,0,6.128051,8.794787,277.217448,276.972462,W,W,0.992605,-0.121392,2013-06-30
25874,25874,Wind Farm 8,2013-06-30 02:59:00,0.737073,6.437643,-0.603173,9.343421,-0.899572,3,6,...,0,6.465839,9.386626,275.352681,275.499408,W,W,0.995397,-0.095835,2013-06-30
25875,25875,Wind Farm 8,2013-06-30 03:59:00,0.381433,5.782137,-0.883416,8.738048,-1.229385,4,6,...,0,5.849233,8.824107,278.686686,278.008567,W,W,0.990247,-0.139321,2013-06-30
25876,25876,Wind Farm 8,2013-06-30 04:59:00,0.669668,5.628645,-1.468735,8.417481,-2.157050,5,6,...,0,5.817115,8.689468,284.624619,284.373229,WNW,WNW,0.968699,-0.248237,2013-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42907,42907,Wind Farm 8,2013-12-31 19:59:00,0.706422,-1.494570,-6.109682,-2.186179,-11.608958,20,12,...,0,6.289830,11.813014,13.745934,10.664941,N,NNE,-0.185065,-0.982726,2013-12-31
42908,42908,Wind Farm 8,2013-12-31 20:59:00,0.706422,4.203471,-6.960113,7.138662,-12.157225,21,12,...,0,8.130949,14.098178,328.870663,329.578756,NNW,NNW,0.506354,-0.862326,2013-12-31
42909,42909,Wind Farm 8,2013-12-31 21:59:00,0.706422,4.491284,-3.568278,8.018306,-6.013379,22,12,...,0,5.736222,10.022672,308.466775,306.868297,NW,NW,0.800017,-0.599978,2013-12-31
42910,42910,Wind Farm 8,2013-12-31 22:59:00,0.706422,2.985547,-3.832448,5.983437,-6.298804,23,12,...,0,4.858102,8.687719,322.080758,316.470841,NW,NW,0.688724,-0.725024,2013-12-31


In [126]:
## observed values
data_all['TIMESTAMP'] = data_all['TIMESTAMP'].apply(lambda x: x-delta_minute)

In [127]:
data_all['DATE'] = data_all['TIMESTAMP'].dt.date

In [128]:
data_all_mean = data_all.groupby('DATE').mean()['TARGETVAR']

In [129]:
df_all_mean = pd.DataFrame(data_all_mean)

In [130]:
df_all_mean.columns = ['TARGETVAR_OBSERVED']
df_all_mean

,TARGETVAR_OBSERVED
DATE,
2013-06-30,0.731638
2013-07-01,0.913873
2013-07-02,0.679659
2013-07-03,0.464416
2013-07-04,0.688612
...,...
2013-12-27,0.399591
2013-12-28,0.478140
2013-12-29,0.050863


In [131]:
df_comparison = df_all_mean.merge(df_forecast_mean, how='inner', on='DATE')

In [132]:
df_comparison.columns = ['TARGETVAR_OBSERVED', 'TARGETVAR_PREDICTED']
df_comparison

,TARGETVAR_OBSERVED,TARGETVAR_PREDICTED
DATE,,
2013-07-01,0.913873,0.600115
2013-07-02,0.679659,0.563627
2013-07-03,0.464416,0.584431
2013-07-04,0.688612,0.732470
2013-07-05,0.815700,0.724957
...,...,...
2013-12-27,0.399591,0.299677
2013-12-28,0.478140,0.380428
2013-12-29,0.050863,0.091375


In [133]:

fig = go.Figure()

fig.add_traces(
    go.Scatter(x=df_comparison.index, y = df_comparison['TARGETVAR_OBSERVED'], 
        mode = 'lines', line=dict(color='red',), name='observed',
    )
)

fig.add_traces(
    go.Scatter(x=df_comparison.index, y = df_comparison['TARGETVAR_PREDICTED'], 
        mode = 'lines', line=dict(color='blue',), name='predicted',
    )
)